In [9]:
#Derive EARTH ENGINE (Sentinel 2) NDVI files with calculation of C factors loop following Kleijn et al. 2000
import arcpy
import csv
with arcpy.EnvManager(overwriteOutput=True, compression="'LZW' 75"):
    for Year in range(2016,2020):
        LCMIN_Rast = "LCM"+ str(Year) +"PlusCrops.tif"
        Folder = "D:\\Data\Dropbox\\Lactuca\Projects\\AgLand SDR\\Data\\VEGNDVISentinel2\\NDVIMaps"
        if Year == 2016:
            Dates = ["2016-12-02", "2016-01-01","2016-01-17","2016-02-02","2016-02-18","2016-03-05","2016-03-21",\
                     "2016-04-06","2016-05-08","2016-04-22","2016-05-24","2016-06-09","2016-06-25","2016-07-11",\
                     "2016-07-27","2016-08-12","2016-08-28","2016-09-13","2016-09-29","2016-10-15","2016-10-31","2016-11-16"]
        elif Year == 2017:    
            Dates = ["2017-01-01","2017-01-17","2017-02-02","2017-02-18","2017-03-06","2017-03-22","2017-04-07",\
                      "2017-04-23","2017-05-09","2017-05-25","2017-06-10","2017-06-26","2017-07-12","2017-07-28",\
                      "2017-08-13","2017-08-29","2017-09-14","2017-09-30","2017-10-16","2017-11-01","2017-11-17","2017-12-03"]
        elif Year == 2018:     
            Dates = ["2018-01-01","2018-01-17","2018-02-02","2018-02-18","2018-03-06","2018-03-22","2018-04-07",\
                     "2018-04-23","2018-05-09","2018-05-25","2018-06-10","2018-06-26","2018-07-12","2018-07-28",\
                     "2018-08-13","2018-08-29","2018-09-14","2018-09-30","2018-10-16","2018-11-01","2018-11-17","2018-12-03"]
        elif Year == 2019:   
            Dates = ["2019-01-17","2019-02-02","2019-02-18","2019-03-06","2019-03-22","2019-04-07","2019-04-23",\
                     "2019-05-09","2019-05-25","2019-06-10","2019-06-26","2019-07-12","2019-07-28","2019-08-13",\
                     "2019-08-29","2019-09-14","2019-09-30","2019-10-16","2019-11-01","2019-11-17","2019-12-03","2019-01-01"]
        else:
            print("Unknown year")
        for x in range(0,22):
            f = ["ValueRast", "ClipRast", "Lookup_Rast", "Minus_Rast", "Divide_Rast", "Times_Rast", "Zonalrast","CfactorRast"]
            for i in f:
                arcpy.Delete_management(i)
            Message = str(Year) + "_" + Dates[x] + "   "
            print(Message, end="\r") 
            InFile = Folder + "\\" + "NDVI_Date-" + Dates[x] +".tif"
            # Outfile 1 and 2 are not written because they take up too much space (1.2 GB compressed)
            #OutFile1 = Folder + "\\" + "NDVI_Date-" + Dates[x] +"_Cleaned.tif"
            #OutFile2 = Folder + "\\" + "C_Value_Day_-" + Dates[x] +".tif"
            OutFile3 = Folder + "\\" + "Landuse_C_Value_" + Dates[x] +".csv"
            # Make sure the shapes are equal and remove water
            arcpy.management.Clip(InFile, "-387.671799999662 7464.66829999909 655330.6656 1218623.9729", "ClipRast",\
                                  "UK_Outline","-999", "ClippingGeometry", "MAINTAIN_EXTENT")
            out_raster = arcpy.sa.SetNull(LCMIN_Rast, "ClipRast", "BHT = '13. Saltwater' Or BHT = '14. Freshwater'");
            out_raster.save("ValueRast")
            out_raster = arcpy.sa.Con("ValueRast", 0, "ValueRast", "VALUE < 0")
            out_raster.save("Lookup_Rast")
            #out_raster.save(OutFile1)
            
            # Transition from NDVI to cover managment factors follow Kleijn et al. 2000
            out_raster = arcpy.sa.Minus(1, "Lookup_Rast"); out_raster.save("Minus_Rast")
            out_raster = arcpy.sa.Divide("Lookup_Rast", "Minus_Rast"); out_raster.save("Divide_Rast")
            out_raster = arcpy.sa.Times(-2, "Divide_Rast"); out_raster.save("Times_Rast")
            out_raster = arcpy.sa.Exp("Times_Rast");
            out_raster.save("CfactorRast")
            #out_raster.save(OutFile2)
            
            # Extract per crop type and save
            with arcpy.EnvManager(cellSize= 25):
                arcpy.sa.ZonalStatisticsAsTable(LCMIN_Rast, "Value", "CfactorRast", "Zonalrast", "DATA", "ALL",\
                                                "CURRENT_SLICE", [90], "AUTO_DETECT", "ARITHMETIC", 360)
            table = "Zonalrast" 
            fields = arcpy.ListFields(table)  
            field_names = [field.name for field in fields]  
            with open(OutFile3,'w') as f:  
                w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
                w.writerow(field_names)  
                for mow in arcpy.SearchCursor(table):  
                    field_vals = [mow.getValue(field.name) for field in fields]  
                    w.writerow(field_vals)
f = ["ValueRast", "ClipRast", "Lookup_Rast", "Minus_Rast", "Divide_Rast", "Times_Rast", "Zonalrast","CfactorRast", table]
for i in f:
    arcpy.Delete_management(i)

In [3]:
#Derive MODIS NDVI files with calculation of C factors loop following Kleijn et al. 2000
import arcpy
import csv
with arcpy.EnvManager(overwriteOutput=True):
   Year = "2018"
   LCMIN_Rast = "LCM"+ Year +"PlusCrops.tif"
   Folder = "D:\Data\Dropbox\Lactuca\Projects\AgLand SDR\Data\VEGNDVIMODIS\VegNDVI"+ Year
   InBase1 = Folder + "\\" + "MOD13Q1.061__250m_16_days_NDVI_doy" +Year
   OutBase1 = Folder + "\\" + "Day_"
   OutBase2 = Folder + "\\" "C_Value_Day_"
   OutBase3 = Folder + "\\" "Landuse_C_Value_"
   Numbers = ["001","017","033","049","065","081","097","113","129","145","161","177",\
              "193","209","225","241","257","273","289","305","321","337","353"]
   for x in range(17,18):
        f = ["ValueRast", "ClipRast", "Lookup_Rast", "Minus_Rast", "Divide_Rast", "Times_Rast", "Zonalrast"]
        for i in f:
            arcpy.Delete_management(i)
        Message = Year + "_" + Numbers[x] + "   "
        print(Message, end="\r") 
        Infile = InBase1 + Numbers[x]+ "_aid0001.tif"
        OutFile1 = OutBase1 + Numbers[x] + "_" + Year +"_Cleaned.tif"
        OutFile2 = OutBase2 + Numbers[x] + "_" + Year +".tif"
        OutFile3 = OutBase3 + Numbers[x] + "_" + Year +".csv"
        #Make sure the shapes are equal and remove water
        arcpy.management.Clip(Infile, "-387.671799999662 7464.66829999909 655330.6656 1218623.9729", "ClipRast",\
                                  "UK_Outline","-999", "ClippingGeometry", "MAINTAIN_EXTENT")
        out_raster = arcpy.sa.SetNull(LCMIN_Rast, "ClipRast", "BHT = '13. Saltwater' Or BHT = '14. Freshwater'");
        out_raster.save("ValueRast")
        out_raster = arcpy.sa.Con(r"ValueRast", 0, r"ValueRast", "VALUE < 0")
        out_raster.save("ValueRast")
        out_raster.save(OutFile1)
        arcpy.management.AddField("ValueRast", "Value32", "DOUBLE", 10, 5, None, '', "NULLABLE", "NON_REQUIRED", '') 
        arcpy.management.CalculateField("ValueRast", "Value32", "!Value!/10000", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
        #Transition from NDVI to cover managment factors follow Kleijn et al. 2000
        out_raster = arcpy.sa.Lookup("ValueRast", "Value32"); out_raster.save(r"Lookup_Rast")
        out_raster = arcpy.sa.Minus(1, "Lookup_Rast"); out_raster.save(r"Minus_Rast")
        out_raster = arcpy.sa.Divide("Lookup_Rast", "Minus_Rast"); out_raster.save(r"Divide_Rast")
        out_raster = arcpy.sa.Times(-2, "Divide_Rast"); out_raster.save(r"Times_Rast")
        out_raster = arcpy.sa.Exp("Times_Rast");
        out_raster.save(OutFile2)
        # Extract per crop type and save
        with arcpy.EnvManager(cellSize= 25):
                arcpy.sa.ZonalStatisticsAsTable(LCMIN_Rast, "value", OutFile2, "Zonalrast", "DATA", "ALL",\
                                                "CURRENT_SLICE", [90], "AUTO_DETECT", "ARITHMETIC", 360)
        table = "Zonalrast" 
        fields = arcpy.ListFields(table)  
        field_names = [field.name for field in fields]  
        with open(OutFile3,'w') as f:  
           w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
           w.writerow(field_names)  
           for mow in arcpy.SearchCursor(table):  
               field_vals = [mow.getValue(field.name) for field in fields]  
               w.writerow(field_vals)